In [ ]:
# (c) 2019 The Johns Hopkins University Applied Physics Laboratory LLC (JHU/APL).
# All Rights Reserved. This material may be only be used, modified, or reproduced
# by or for the U.S. Government pursuant to the license rights granted under the
# clauses at DFARS 252.227-7013/7014 or FAR 52.227-14. For any other permission,
# please contact the Office of Technology Transfer at JHU/APL.

# NO WARRANTY, NO LIABILITY. THIS MATERIAL IS PROVIDED “AS IS.” JHU/APL MAKES NO
# REPRESENTATION OR WARRANTY WITH RESPECT TO THE PERFORMANCE OF THE MATERIALS,
# INCLUDING THEIR SAFETY, EFFECTIVENESS, OR COMMERCIAL VIABILITY, AND DISCLAIMS
# ALL WARRANTIES IN THE MATERIAL, WHETHER EXPRESS OR IMPLIED, INCLUDING (BUT NOT
# LIMITED TO) ANY AND ALL IMPLIED WARRANTIES OF PERFORMANCE, MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE, AND NON-INFRINGEMENT OF INTELLECTUAL PROPERTY
# OR OTHER THIRD PARTY RIGHTS. ANY USER OF THE MATERIAL ASSUMES THE ENTIRE RISK
# AND LIABILITY FOR USING THE MATERIAL. IN NO EVENT SHALL JHU/APL BE LIABLE TO ANY
# USER OF THE MATERIAL FOR ANY ACTUAL, INDIRECT, CONSEQUENTIAL, SPECIAL OR OTHER
# DAMAGES ARISING FROM THE USE OF, OR INABILITY TO USE, THE MATERIAL, INCLUDING,
# BUT NOT LIMITED TO, ANY DAMAGES FOR LOST PROFITS.

In [ ]:
import fnmatch
import json
import pandas as pd
from itertools import product
from scipy import stats
from pathlib import Path

pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
metrics_dict = {'perf_recovery': ['alternating'],
                'perf_maintenance': ['alternating', 'permuted'],
                'forward_transfer_ratio': ['permuted'],
                'backward_transfer_ratio': ['permuted'],
                'forward_transfer_contrast': ['permuted'],
                'backward_transfer_contrast': ['permuted'],
                'ste_rel_perf': ['alternating', 'permuted'],
                'sample_efficiency': ['alternating', 'permuted']}

In [ ]:
# Aggregate tsv data into single table
sg_metrics_df = pd.DataFrame()

# Import saved data
root_dir = Path('results')
run_types = ['raw', 'smoothed', 'normalized', 'normalized_no_outliers']
sg_names = ['argonne', 'hrl', 'sri', 'teledyne', 'upenn']

for sg_name, run_type in product(sg_names, run_types):
    metrics_file = root_dir / run_type / sg_name / (sg_name + '_' + run_type + '.tsv')
    if metrics_file.exists():
        ll_metrics_df = pd.read_csv(metrics_file, sep='\t')
        ll_metrics_df['run_type'] = run_type
        sg_metrics_df = sg_metrics_df.append(ll_metrics_df)

# Rename transfer columns
sg_metrics_df.rename(columns={'perf_maintenance_mrlep': 'perf_maintenance'}, inplace=True)
sg_metrics_df

In [ ]:
metric_dfs = []

for metric in metrics_dict.keys():
    metric_dfs.append(sg_metrics_df[sg_metrics_df['scenario_type'].isin(metrics_dict[metric])].groupby(by=['run_type', 'sg_name', 'agent_config', 'complexity'])[metric].agg(['mean']).rename({'mean': metric}, axis=1))

In [ ]:
results_df = pd.concat(metric_dfs, axis=1).reset_index().drop(columns=['agent_config', 'complexity'])
results_df

In [ ]:
# Save results to file
# sg_metrics_df.sort_values(by=['run_type', 'sg_name']).to_csv('sg_metrics.tsv', sep='\t')

In [ ]:
# Create target values dataframe
target_values = [[1.2, 10, 4, 2.5, 2, 2.3],
                 [1, 4, 1.5, 1.5, 1.6, 2],
                 [0.6, 6, 3, 2, 1.9, 2.4]]

metrics = ['perf_recovery', 'perf_maintenance', 'forward_transfer',
           'backward_transfer', 'ste_rel_perf', 'sample_efficiency']
indeces = pd.Index(['1-low', '2-intermediate', '3-high'], name='complexity')
target_values_df = pd.DataFrame(data=target_values, columns=metrics, index=indeces)
target_values_df